In [9]:
import os

In [11]:
os.chdir("../")

In [12]:
os.getcwd()

'd:\\Dropbox\\Self-Development\\Coding_Projects\\TextSummarizer'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    input_feature: str
    target_feature: str

In [14]:
from src.constants import *
from src.utils.exception import CustomException
from src.utils.logger import logger
from src.utils.utils import load_yaml, create_directories
#from src.entity import DataTransformationConfig


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = load_yaml(config_filepath)
        self.params = load_yaml(params_filepath)

        create_directories([self.config["artifacts_root"]])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config["data_transformation"]

        create_directories([config["root_dir"]])

        data_transformation_config = DataTransformationConfig(
            root_dir=config["root_dir"],
            data_path=config["data_path"],
            tokenizer_name=config["tokenizer_name"],
            input_feature=config["input_feature"],
            target_feature=config["target_feature"]
        )

        return data_transformation_config

In [17]:
import os
import sys
from src.utils.exception import CustomException
from src.utils.logger import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    def convert_examples_to_features(self, batch):
        input_encodings = self.tokenizer(batch[self.config.input_feature], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(batch[self.config.target_feature], max_length=1024, truncation=True)
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
        
    def convert(self):
        try:
            dataset = load_from_disk(self.config.data_path)
            dataset_transformed = dataset.map(self.convert_examples_to_features, batched=True)
            logger.info("Dataset transformation done.")
            dataset_transformed.save_to_disk(os.path.join(self.config.root_dir, 'transformed_dataset'))
            logger.info("Transformed dataset saved")
        except Exception as e:
            raise CustomException(e, sys)

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise CustomException(e, sys)

[2023-08-06 23:49:26,682: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-08-06 23:49:26,685: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-08-06 23:49:26,686: INFO: utils: created directory at: artifacts]
[2023-08-06 23:49:26,687: INFO: utils: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/819 [00:00<?, ? examples/s]d:\Dropbox\Self-Development\Coding_Projects\TextSummarizer\venv\lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 819/819 [00:00<00:00, 5532.03 examples/s]


[2023-08-06 23:49:27,495: INFO: 131666484: Dataset transformation done.]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 102220.85 examples/s]

[2023-08-06 23:49:27,560: INFO: 131666484: Transformed dataset saved]


In [19]:
data = load_from_disk('artifacts/data_transformation/transformed_dataset')
data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})